In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import graphviz
from graphviz import Digraph
#https://stackoverflow.com/questions/52472611/how-do-i-make-sure-the-graphviz-executables-are-on-my-systems-path
os.environ["PATH"] += os.pathsep + r'C:/Program Files/Graphviz/bin/'
%matplotlib inline

In [2]:
def f(x):
    return 3*x**2 - 4*x + 5

In [3]:
f(3.0)

20.0

In [4]:
xs = np.arange(-5,5,0.25)
ys = f(xs)
ys

array([100.    ,  91.6875,  83.75  ,  76.1875,  69.    ,  62.1875,
        55.75  ,  49.6875,  44.    ,  38.6875,  33.75  ,  29.1875,
        25.    ,  21.1875,  17.75  ,  14.6875,  12.    ,   9.6875,
         7.75  ,   6.1875,   5.    ,   4.1875,   3.75  ,   3.6875,
         4.    ,   4.6875,   5.75  ,   7.1875,   9.    ,  11.1875,
        13.75  ,  16.6875,  20.    ,  23.6875,  27.75  ,  32.1875,
        37.    ,  42.1875,  47.75  ,  53.6875])

In [5]:
h = 0.001
x = -3.0
(f(x + h) - f(x))/h

-21.996999999998934

In [6]:
a = 2.0
b = -3.0
c = 10.0
d = a*b + c
d

4.0

In [7]:
h = 0.0001

a = 2.0
b = -3.0
c = 10.0

d1 = a*b + c
a += h
d2 = a*b + c

print('slope',(d2 - d1)/h)

slope -3.000000000010772


In [8]:
h = 0.0001

a = 2.0
b = -3.0
c = 10.0

d1 = a*b + c
b += h
d2 = a*b + c

print('slope',(d2 - d1)/h)

slope 2.0000000000042206


In [9]:
h = 0.0001

a = 2.0
b = -3.0
c = 10.0

d1 = a*b + c
c += h
d2 = a*b + c

print('slope',(d2 - d1)/h)

slope 0.9999999999976694


Building the Value Object

In [10]:
class Value:
    def __init__(self,data):
        self.data = data
    def __repr__(self):
        return f"Value(data={self.data})"
    def __add__(self,other):
        out = Value(self.data + other.data)
        return out
    def __mul__(self,other):
        out = Value(self.data * other.data)
        return out

a = Value(2.0)
b = Value(-3.0)
c = Value(10)
a + b
a * b
a*b + c

Value(data=4.0)

The above implementation cannot maintain a computation graph. Hence its required that each node/value keep track of which values hold data and connected to it

In [11]:
class Value:
    #when we create a single values Value object, we dont
    #pass in any children, however when we do an operation
    #we pass in the associated objects to be held as children
    def __init__(self,data,_children = ()):
        #the input will be a tuple of children
        #but the data will be held as a Set
        #done for efficiency
        self.data = data
        self._prev = set(_children)
    def __repr__(self):
        return f"Value(data={self.data})"
    def __add__(self,other):
        out = Value(self.data + other.data,_children = (self,other))
        return out
    def __mul__(self,other):
        out = Value(self.data * other.data,_children = (self,other))
        return out
    
a = Value(2.0)
b = Value(-3.0)
c = Value(10)
a + b
a * b
#a*b results in a Value object with _prev
#The object from a*b is fed into the addition method
#and the final object will also have a _prev
d = a*b + c
d

Value(data=4.0)

In [12]:
d._prev

{Value(data=-6.0), Value(data=10)}

Currently we still dont know which operation created the d value

In [13]:
class Value:
    #when we create a single values Value object, we dont
    #pass in any children, however when we do an operation
    #we pass in the associated objects to be held as children
    def __init__(self,data,_children = (),_op = '',label = ''):
        #the input will be a tuple of children
        #but the data will be held as a Set
        #done for efficiency
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.label = label
    def __repr__(self):
        return f"Value(data={self.data})"
    def __add__(self,other):
        out = Value(self.data + other.data,_children = (self,other),_op = '+')
        return out
    def __mul__(self,other):
        out = Value(self.data * other.data,_children = (self,other),_op = '*')
        return out

a = Value(2.0,label = 'a')
b = Value(-3.0,label = 'b')
c = Value(10,label = 'c')
a + b
a * b
d = a*b + c
d


Value(data=4.0)

In [22]:
d._prev

{Value(data=-6.0), Value(data=10)}

In [23]:
for i in d._prev:
    print(i._prev)

{Value(data=2.0), Value(data=-3.0)}
set()


The above expressions are simple. But they get large very fast. Hence we need a way to visualize these functions. 

In [26]:
def trace(root):
    nodes,edges = set(),set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
        for i in v._prev:
            edges.add((v,i,v._op))
            build(i)
    build(root)
    return nodes,edges

nodes,edges = trace(d)
#26:55



In [27]:
nodes

{Value(data=-3.0),
 Value(data=-6.0),
 Value(data=10),
 Value(data=2.0),
 Value(data=4.0)}

In [28]:
edges

{(Value(data=-6.0), Value(data=-3.0), '*'),
 (Value(data=-6.0), Value(data=2.0), '*'),
 (Value(data=4.0), Value(data=-6.0), '+'),
 (Value(data=4.0), Value(data=10), '+')}